In [1]:
import openai

In [2]:
from openai import OpenAI

In [3]:
client = OpenAI()

In [4]:
import os

In [7]:
openai.api_key = os.getenv('OPENAI_TOKEN')

In [8]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": "is it too late to join the course?"}]
)

In [9]:
response.choices[0].message.content

"It depends on the specific course and its enrollment policies. Some courses may have rolling admissions or allow students to join at any time, while others may have strict deadlines. Here are a few steps you can take to find out if it’s too late to join:\n\n1. **Check the Course Website:** Look for any information regarding enrollment deadlines or late registration policies.\n2. **Contact the Instructor or Administration:** Reach out to the course instructor or the administrative office handling registrations. They can provide you with the most accurate information regarding enrollment.\n3. **Look for Prerequisites and Course Start Dates:** Verify if there are prerequisites you need to meet before joining and check the start dates to see if you’ve missed any critical information or classes.\n4. **Explore Alternative Options:** If it's too late to join the current session, ask if there’s a possibility to join a future session or course offering.\n\nTaking these steps will give you a cl